In [1]:
# 排除 淡水河、磺溪、後龍溪、烏溪、朴子溪、高屏溪、花蓮溪等圖框編號
# 將剩餘圖框之土地利用的數字轉換成int datatype
# rasterize landuse number from shp to tif 

import glob
import os
from pathlib import Path

targetPattern = r"F:\intern\Landuse_EPSG3826\original\*.shp"
full_path = glob.glob(targetPattern) # 返回結果是匹配給定模式的帶完整路徑的檔名

shp_list = [] # 儲存所有shp名稱的空清單

for i in full_path:
    shp = Path(i).stem # 從完整路徑篩選出shp檔名(沒有副檔名)
    shp_list.append(shp)
    

shp_list = list(map(int, shp_list)) # 將lsit裡面的 datatype change from str to int
len(shp_list) # 資料夾內shp個數

5287

In [2]:
import pandas as pd

#讀取九大流域的shp資料
nine_basin = pd.read_csv('F:\intern\Landuse_EPSG3826\9basin_Picture frame.csv')
nine_shp_list = nine_basin['圖幅_5K'].tolist() # 九大流域shp名稱

left_shp = set(shp_list).difference(nine_shp_list) # 從 shp_lsit 中移除 nine_shp_list 已有的圖框
left_shp # 未與九大流域重複的圖框

{97223060,
 97223085,
 97223086,
 97223087,
 97223088,
 97223089,
 97223090,
 97223095,
 97223096,
 97223097,
 97223098,
 97223099,
 97223100,
 94192056,
 94192057,
 94192058,
 94192059,
 94192067,
 94192068,
 94192077,
 94192078,
 94192088,
 94192098,
 94193006,
 94193007,
 96191001,
 96191002,
 96191003,
 96191004,
 96191005,
 96191006,
 96191007,
 96191008,
 96191009,
 96191010,
 96191011,
 96191012,
 96191013,
 96191014,
 96191015,
 96191016,
 96191017,
 96191018,
 96191019,
 96191020,
 96191021,
 96191022,
 96191023,
 96191024,
 96191025,
 96191026,
 96191027,
 96191028,
 96191029,
 96191030,
 96191031,
 96191032,
 96191033,
 96191034,
 96191035,
 96191036,
 96191037,
 96191038,
 96191039,
 96191040,
 96191041,
 96191042,
 96191043,
 96191044,
 96191045,
 96191046,
 96191047,
 96191048,
 96191049,
 96191051,
 96191052,
 96191053,
 96191054,
 96191055,
 96191056,
 96191057,
 96191058,
 96191059,
 94193010,
 96191061,
 96191062,
 96191063,
 96191064,
 96191065,
 96191066,
 96191067,

In [7]:
import geopandas as gpd
shp_path = r'F:\intern\Landuse_EPSG3826\original\\' 

for i in left_shp:
    p = shp_path + str(i) + '.shp' # shp 讀檔路徑
    
    gdf_i = gpd.read_file(p) # 第i個shp存成 gdf_i
    
    value=[] # 存放土地利用編號
    
    for j in range(len(gdf_i['LCODE_C1'])): # 每個 shp 逐行讀取
        try:
            v = int(gdf_i.iloc[j,3]) # 將 str 轉換成 int
            value.append(v)
            
        except Exception as e:
            v=int(4)   #有問題先假設水域
            value.append(v)

    gdf_i['value']=value
    
    gdf_i.to_file("F:\\intern\\Landuse_EPSG3826\\converted\\" + str(i) + ".shp")
    
    

In [ ]:
# 土地利用 batch轉換 (shp to tif)
# 使用 OSGGeo4w shell 執行以下程式碼
for %i in (F:\intern\Landuse_EPSG3826\converted\*.shp) DO gdal_rasterize -a value -of GTiff -a_srs EPSG:3826 -co "COMPRESS=DEFLATE" -a_nodata -9999 -tr 1 1 -ot Float32 %i F:\intern\Landuse_EPSG3826\processed\%~ni.tif